# Project 3: **Ask‑the‑Web Agent**

Welcome to Project 3! In this project, you will learn how to use tool‑calling LLMs, extend them with custom tools, and build a simplified *Perplexity‑style* agent that answers questions by searching the web.

## Learning Objectives  
* Understand why tool calling is useful and how LLMs can invoke external tools.
* Implement a minimal loop that parses the LLM's output and executes a Python function.
* See how *function schemas* (docstrings and type hints) let us scale to many tools.
* Use **LangChain** to get function‑calling capability for free (ReAct reasoning, memory, multi‑step planning).
* Combine LLM with a web‑search tool to build a simple ask‑the‑web agent.

## Roadmap
1. Environment setup
2. Write simple tools and connect them to an LLM
3. Standardize tool calling by writing `to_schema`
4. Use LangChain to augment an LLM with your tools
5. Build a Perplexity‑style web‑search agent
6. (Optional) A minimal backend and frontend UI

# 1- Environment setup

## 1.1- Conda environment

Before we start coding, you need a reproducible setup. Open a terminal in the same directory as this notebook and run:

```bash
# Create and activate the conda environment
conda env create -f environment.yml && conda activate web_agent

# Register this environment as a Jupyter kernel
python -m ipykernel install --user --name=web_agent --display-name "web_agent"
```
Once this is done, you can select “web_agent” from the Kernel → Change Kernel menu in Jupyter or VS Code.


> Behind the scenes:
> * Conda reads `environment.yml`, resolves the pinned dependencies, creates an isolated environment named `web_agent`, and activates it.
> * `ollama pull` downloads the model so you can run it locally without API calls.


## 1.2 Ollama setup

In this project, we start with `gemma3-1B` because it is lightweight and runs on most machines. You can try other smaller or larger LLMs such as `mistral:7b`, `phi3:mini`, or `llama3.2:1b` to compare performance. Explore available models here: https://ollama.com/library

```bash
ollama pull gemma3:1b
```

`ollama pull` downloads the model so you can run it locally without API calls.


## 2- Tool Calling

LLMs are strong at answering questions, but they cannot directly access external data such as live web results, APIs, or computations. In real applications, agents rarely rely only on their internal knowledge. They need to query APIs, retrieve data, or perform calculations to stay accurate and useful. Tool calling bridges this gap by allowing the LLM to request actions from the outside world.


We describe each tool’s interface in the model’s prompt, defining what it does and what arguments it expects. When the model decides that a tool is needed, it emits a structured output like: `TOOL_CALL: {"name": "get_current_weather", "args": {"city": "San Francisco"}}`. Your code will detect this output, execute the corresponding function, and feed the result back to the LLM so the conversation continues.

In this section, you will implement a simple `get_current_weather` function and teach the `gemma3` model how to use it when required in four steps:
1. Implement the tool
2. Create the instructions for the LLM
3. Call the LLM with the prompt
4. Parse the LLM output and call the tool

In [11]:
from openai import OpenAI

client = OpenAI(api_key = "ollama", base_url = "http://host.docker.internal:11434/v1")

In [2]:
# ---------------------------------------------------------
# Step 1: Implement the tool
# ---------------------------------------------------------
# Your goal: give the model a way to access weather information.
# You can either:
#   (a) Call a real weather API (for example, OpenWeatherMap), or
#   (b) Create a dummy function that returns a fixed response (e.g., "It is 23°C and sunny in San Francisco.")
#
# Requirements:
#   • The function should be named `get_current_weather`
#   • It should take two arguments:
#         - city: str
#         - unit: str = "celsius"
#   • Return a short, human-readable sentence describing the weather.
#
# Example expected behavior:
#   get_current_weather("San Francisco") → "It is 23°C and sunny in San Francisco."
#

def get_current_weather(city: str, unit: str = "celsius") -> str:
    """Get the current weather for a city. Returns a human-readable weather description."""
    return f"It is 23°C and sunny in {city}."

In [3]:
# ---------------------------------------------------------
# Step 2: Create the prompt for the LLM to call tools
# ---------------------------------------------------------
# Goal:
#   Build the system and user prompts that instruct the model when and how
#   to use your tool (`get_current_weather`).
#
# What to include:
#   • A SYSTEM_PROMPT that tells the model about the tool use and describe the tool
#   • A USER_QUESTION with a user query that should trigger the tool.
#       Example: "What is the weather in San Diego today?"

# Try experimenting with different system and user prompts
# ---------------------------------------------------------

SYSTEM_PROMPT = """You are a helpful assistant with access to tools.

When you need to use a tool, respond with the following format:
TOOL_CALL: {"name": "tool_name", "args": {"arg1": "value1", "arg2": "value2"}}

Available tools:
- get_current_weather: Get the current weather for a city
  Arguments:
    - city (str): The name of the city
    - unit (str, optional): Temperature unit, either "celsius" or "fahrenheit". Default is "celsius"

Only use TOOL_CALL when you need weather information. Otherwise, respond normally.
"""

USER_QUESTION_1 = "What is the weather in San Diego today?"
USER_QUESTION_2 = "Will I need a jacket in Tokyo?"

Now that you have defined a tool and shown the model how to use it, the next step is to call the LLM using your prompt.

Start the **Ollama** server in a terminal with `ollama serve`. This launches a local API endpoint that listens for LLM requests. Once the server is running, return to the notebook and in the next cell send a query to the model.


In [12]:
# ---------------------------------------------------------
# Step 3: Call the LLM with your prompt
# ---------------------------------------------------------
# Task:
#   Send SYSTEM_PROMPT + USER_QUESTION to the model.
#
# Steps:
#   1. Use the Ollama client to create a chat completion. 
#       - You may find some examples here: https://platform.openai.com/docs/api-reference/chat/create
#       - If you are unsure, search the web for "client.chat.completions.create"
#   2. Print the raw response.
#
# Expected:
#   The model should return something like:
#   TOOL_CALL: {"name": "get_current_weather", "args": {"city": "San Diego"}}
# ---------------------------------------------------------

# Test with USER_QUESTION_1
response_1 = client.chat.completions.create(
    model="gemma3:4b",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_QUESTION_1}
    ],
    temperature=0
)

print("=" * 60)
print(f"USER QUESTION 1: {USER_QUESTION_1}")
print("=" * 60)
print(f"MODEL RESPONSE:\n{response_1.choices[0].message.content}")
print("\n")

# Test with USER_QUESTION_2
response_2 = client.chat.completions.create(
    model="gemma3:4b",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_QUESTION_2}
    ],
    temperature=0
)

print("=" * 60)
print(f"USER QUESTION 2: {USER_QUESTION_2}")
print("=" * 60)
print(f"MODEL RESPONSE:\n{response_2.choices[0].message.content}")

USER QUESTION 1: What is the weather in San Diego today?
MODEL RESPONSE:
TOOL_CALL: {"name": "get_current_weather", "args": {"city": "San Diego"}}


USER QUESTION 2: Will I need a jacket in Tokyo?
MODEL RESPONSE:
To figure that out, I need to check the weather in Tokyo.

TOOL_CALL: {"name": "get_current_weather", "args": {"city": "Tokyo", "unit": "celsius"}}


In [15]:
# ---------------------------------------------------------
# Step 4: Parse the LLM output and call the tool
# ---------------------------------------------------------
# Task:
#   Detect when the model requests a tool, extract its name and arguments,
#   and execute the corresponding function.
#
# Steps:
#   1. Search for the text pattern "TOOL_CALL:{...}" in the model output.
#   2. Parse the JSON inside it to get the tool name and args.
#   3. Call the matching function (e.g., get_current_weather).
#
# Expected:
#   You should see a line like:
#       Calling tool `get_current_weather` with args {'city': 'San Diego'}
#       Result: It is 23°C and sunny in San Diego.
# ---------------------------------------------------------

import re, json

def parse_and_execute_tool(response_text: str) -> str:
    """
    Parse the LLM response for TOOL_CALL and execute the corresponding function.
    
    Args:
        response_text: The raw text response from the LLM
        
    Returns:
        The result of the tool call, or the original response if no tool was called
    """
    # Search for the TOOL_CALL pattern with proper JSON capture
    match = re.search(r'TOOL_CALL:\s*(\{.*?\}\s*\})', response_text, re.DOTALL)
    
    if match:
        try:
            # Extract and parse the JSON
            tool_call_json = match.group(1)
            print(f"Calling tool...")
            
            tool_call = json.loads(tool_call_json)
            
            tool_name = tool_call.get("name")
            tool_args = tool_call.get("args", {})
            
            print(f"Tool: `{tool_name}` with args {tool_args}")
            
            # Execute the tool (currently only supports get_current_weather)
            if tool_name == "get_current_weather":
                result = get_current_weather(**tool_args)
                print(f"Result: {result}")
                return result
            else:
                return f"Unknown tool: {tool_name}"
        except json.JSONDecodeError as e:
            print(f"ERROR - Failed to parse JSON: {e}")
            return f"Error parsing tool call: {e}"
    else:
        # No tool call detected, return the original response
        return response_text


# Test with response_1
print("=" * 60)
print("PROCESSING RESPONSE 1")
print("=" * 60)
result_1 = parse_and_execute_tool(response_1.choices[0].message.content)
print("\n")

# Test with response_2
print("=" * 60)
print("PROCESSING RESPONSE 2")
print("=" * 60)
result_2 = parse_and_execute_tool(response_2.choices[0].message.content)

PROCESSING RESPONSE 1
Calling tool...
Tool: `get_current_weather` with args {'city': 'San Diego'}
Result: It is 23°C and sunny in San Diego.


PROCESSING RESPONSE 2
Calling tool...
Tool: `get_current_weather` with args {'city': 'Tokyo', 'unit': 'celsius'}
Result: It is 23°C and sunny in Tokyo.


# 3- Standadize tool calling

So far, we handled tool calling manually by writing one regex and one hard-coded function. This approach does not scale if we want to add more tools. Adding more tools would mean more `if/else` blocks and manual edits to the `TOOL_SPEC` prompt.

To make the system flexible, we can standardize tool definitions by automatically reading each function’s signature, converting it to a JSON schema, and passing that schema to the LLM. This way, the LLM can dynamically understand which tools exist and how to call them without requiring manual updates to prompts or conditional logic.

Next, you will implement a small helper that extracts metadata from functions and builds a schema for each tool.

In [16]:
# ---------------------------------------------------------
# Generate a JSON schema for a tool automatically
# ---------------------------------------------------------
#
# Steps:
#   1. Use `inspect.signature` to get function parameters.
#   2. For each argument, record its name, type, and description.
#   3. Build a schema containing:
#   4. Test your helper on `get_current_weather` and print the result.
#
# Expected:
#   A dictionary describing the tool (its name, args, and types).
# ---------------------------------------------------------

from pprint import pprint
import inspect


def to_schema(fn):
    """
    Convert a Python function to a JSON schema for tool calling.
    Extracts function name, parameters, types, and description from docstring.
    """
    # Get function signature
    sig = inspect.signature(fn)
    
    # Build the schema dictionary
    schema = {
        "name": fn.__name__,
        "description": fn.__doc__.strip() if fn.__doc__ else "",
        "parameters": {}
    }
    
    # Extract parameter information
    for param_name, param in sig.parameters.items():
        param_info = {"type": "string"}  # default type
        
        # Get type annotation if available
        if param.annotation != inspect.Parameter.empty:
            type_name = param.annotation.__name__ if hasattr(param.annotation, '__name__') else str(param.annotation)
            param_info["type"] = type_name
        
        # Get default value if available
        if param.default != inspect.Parameter.empty:
            param_info["default"] = param.default
        
        schema["parameters"][param_name] = param_info
    
    return schema

tool_schema = to_schema(get_current_weather)
pprint(tool_schema)

{'description': 'Get the current weather for a city. Returns a human-readable '
                'weather description.',
 'name': 'get_current_weather',
 'parameters': {'city': {'type': 'str'},
                'unit': {'default': 'celsius', 'type': 'str'}}}


In [17]:
# ---------------------------------------------------------
# Provide the tool schema to the model
# ---------------------------------------------------------
# Goal:
#   Give the model a "menu" of available tools so it can choose
#   which one to call based on the user’s question.
#
# Steps:
#   1. Add an extra system message (e.g., name="tool_spec")
#      containing the JSON schema(s) of your tools.
#   2. Include SYSTEM_PROMPT and the user question as before.
#   3. Send the messages to the model (e.g., gemma3:1b).
#   4. Print the raw model output to see if it picks the right tool.
#
# Expected:
#   The model should produce a structured TOOL_CALL indicating
#   which tool to use and with what arguments.
# ---------------------------------------------------------

import json

# Create a list of available tools with their schemas
available_tools = [get_current_weather]
tool_schemas = [to_schema(fn) for fn in available_tools]

# Create an enhanced system prompt that includes the tool schemas
ENHANCED_SYSTEM_PROMPT = f"""You are a helpful assistant with access to tools.

When you need to use a tool, respond with the following format:
TOOL_CALL: {{"name": "tool_name", "args": {{"arg1": "value1", "arg2": "value2"}}}}

Available tools:
{json.dumps(tool_schemas, indent=2)}

Only use TOOL_CALL when you need to use a tool. Otherwise, respond normally.
"""

# Test with a new question
USER_QUESTION_3 = "What's the weather like in Paris?"

response_3 = client.chat.completions.create(
    model="gemma3:4b",
    messages=[
        {"role": "system", "content": ENHANCED_SYSTEM_PROMPT},
        {"role": "user", "content": USER_QUESTION_3}
    ],
    temperature=0
)

print("=" * 60)
print(f"USER QUESTION: {USER_QUESTION_3}")
print("=" * 60)
print(f"MODEL RESPONSE:\n{response_3.choices[0].message.content}")
print("\n")

# Parse and execute the tool call
print("=" * 60)
print("EXECUTING TOOL CALL")
print("=" * 60)
result_3 = parse_and_execute_tool(response_3.choices[0].message.content)

USER QUESTION: What's the weather like in Paris?
MODEL RESPONSE:
TOOL_CALL: {"name": "get_current_weather", "args": {"city": "Paris", "unit": "celsius"}}


EXECUTING TOOL CALL
Calling tool...
Tool: `get_current_weather` with args {'city': 'Paris', 'unit': 'celsius'}
Result: It is 23°C and sunny in Paris.


## 4- LangChain for Tool Calling
So far, you built a simple tool-calling pipeline manually. While this helps you understand the logic, it does not scale well when working with multiple tools, complex parsing, or multi-step reasoning.

LangChain simplifies this process. You only need to declare your tools, and its *Agent* abstraction handles when to call a tool, how to use it, and how to continue reasoning afterward.

In this section, you will use the **ReAct** Agent (Reasoning + Acting). It alternates between reasoning steps and tool use, producing clearer and more reliable results. We will explore reasoning-focused models in more depth next week.

The following links might be helpful:
- https://python.langchain.com/api_reference/langchain/agents/langchain.agents.initialize.initialize_agent.html
- https://python.langchain.com/docs/integrations/tools/
- https://python.langchain.com/docs/integrations/chat/ollama/
- https://python.langchain.com/api_reference/core/language_models/langchain_core.language_models.llms.LLM.html

In [18]:
# ---------------------------------------------------------
# Step 1: Define tools for LangChain
# ---------------------------------------------------------
# Goal:
#   Convert your weather function into a LangChain-compatible tool.
#
# Steps:
#   1. Import `tool` from `langchain.tools`.
#   2. Keep your existing `get_current_weather` helper as before.
#   3. Create a new function (e.g., get_weather) that calls it.
#   4. Add the `@tool` decorator so LangChain can register it automatically.
#
# Notes:
#   • The decorator converts your Python function into a standardized tool object.
#   • Start with keeping the logic simple and offline-friendly.

from langchain.tools import tool

@tool
def get_weather(city: str, unit: str = "celsius") -> str:
    """Get the current weather for a city. Returns a human-readable weather description.
    
    Args:
        city: The name of the city
        unit: Temperature unit, either 'celsius' or 'fahrenheit'. Default is 'celsius'
    """
    return get_current_weather(city, unit)

In [21]:
# ---------------------------------------------------------
# Step 2: Initialize the LangChain Agent
# ---------------------------------------------------------
# Goal:
#   Connect your tool to a local LLM using LangChain’s ReAct-style agent.
#
# Steps:
#   1. Import the required classes:
#        - ChatOllama (for local model access)
#        - initialize_agent, Tool, AgentType
#   2. Create an LLM instance (e.g., model="gemma3:1b", temperature=0).
#   3. Add your tool(s) to a list
#   4. Initialize the agent using initialize_agent
#   5. Test the agent with a natural question (e.g., "Do I need an umbrella in Seattle today?").
#
# Expected:
#   The model should reason through the question, call your tool,
#   and produce a final answer in plain language.
# ---------------------------------------------------------

from langchain_community.chat_models import ChatOllama
from langchain.agents import initialize_agent, Tool, AgentType

# Create an LLM instance with the correct base URL for dev container
llm = ChatOllama(
    model="gemma3:4b", 
    temperature=0,
    base_url="http://host.docker.internal:11434"
)

# Create tools list
tools = [get_weather]

# Initialize the agent with ReAct reasoning
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# Test the agent using invoke instead of run
response = agent.invoke({"input": "Do I need an umbrella in Seattle today?"})
print("\n" + "=" * 60)
print("FINAL ANSWER:")
print("=" * 60)
print(response["output"])



> Entering new AgentExecutor chain...
Action: ```{
  "action": "get_weather",
  "action_input": {
    "city": "Seattle"
  }
}```
Observation: It is 23°C and sunny in Seattle.
Thought:Action: ```{
  "action": "Final Answer",
  "action_input": "No, it's sunny and 23°C in Seattle today."
}```

> Finished chain.

FINAL ANSWER:
No, it's sunny and 23°C in Seattle today.


### What just happened?

The console log displays the **Thought → Action → Observation → …** loop until the agent produces its final answer. Because `verbose=True`, LangChain prints each intermediate reasoning step.

If you want to add more tools, simply append them to the tools list. LangChain will handle argument validation, schema generation, and tool-calling logic automatically.


## 5- Perplexity‑Style Web Search
Agents become much more powerful when they can look up real information on the web instead of relying only on their internal knowledge.

In this section, you will combine everything you have learned to build a simple Ask-the-Web Agent. You will integrate a web search tool (DuckDuckGo) and make it available to the agent using the same tool-calling approach as before.

This will let the model retrieve fresh results, reason over them, and generate an informed answer—similar to how Perplexity works.

You may find some examples from the following links:
- https://pypi.org/project/duckduckgo-search/

In [23]:
# ---------------------------------------------------------
# Step 1: Add a web search tool
# ---------------------------------------------------------
# Goal:
#   Create a tool that lets the agent search the web and return results.
#
# Steps:
#   1. Use DuckDuckGo for quick, open web searches.
#   2. Write a helper function (e.g., search_web) that:
#        • Takes a query string
#        • Uses DDGS to fetch top results (titles + URLs)
#        • Returns them as a formatted string
#   3. Wrap it with the @tool decorator to make it available to LangChain.


# ---------------------------------------------------------
# Step 1: Add a web search tool
# ---------------------------------------------------------

from ddgs import DDGS
from langchain.tools import tool

@tool
def web_search(query: str) -> str:
    """Search the web using DuckDuckGo and return top results.
    
    Args:
        query: The search query string
        
    Returns:
        A formatted string containing search results with titles, snippets, and URLs
    """
    try:
        # Initialize DuckDuckGo search
        with DDGS() as ddgs:
            # Perform search and get top 5 results
            results = list(ddgs.text(query, max_results=5))
        
        # Format the results into a readable string
        if not results:
            return "No search results found."
        
        formatted_results = []
        for i, result in enumerate(results, 1):
            title = result.get('title', 'No title')
            snippet = result.get('body', 'No description')
            url = result.get('href', 'No URL')
            
            formatted_results.append(
                f"{i}. {title}\n"
                f"   {snippet}\n"
                f"   URL: {url}\n"
            )
        
        return "\n".join(formatted_results)
    
    except Exception as e:
        return f"Error performing web search: {str(e)}"


# Test the search tool
print("Testing web_search tool:")
print("=" * 60)
test_result = web_search.invoke({"query": "latest AI developments 2024"})
print(test_result)

Testing web_search tool:
1. Anticipated AI Developments for 2024
   In summary, 2024 holds exciting prospects for AI , with advancements, ethical considerations, widespread use, and regulatory developments shaping its path. Adapting, learning, and using AI responsibly will be key to unlocking its full potential.
   URL: https://www.linkedin.com/pulse/anticipated-ai-developments-2024-pradeep-patel

2. AI Developments 2024 -2030: Multimodal, AI Agents and Robotics
   2024 – Introductie multimodale taalmodellen. Integratie van AI in bestaande apparaten en applicaties: AI toepassingen worden steeds meer onderdeel van de pparaten en softwarepakketten die we dagelijks gebruiken.
   URL: https://nxtli.com/en/ai-trends-2024-2030-multimodal-ai-agents-robotics/

3. Biggest AI Developments of 2024 So Far and What’s Ahead
   On February 13, 2024 , the USPTO published “Inventorship Guidance for AI -Assisted Inventions” which confirmed that generative AI cannot be an inventor nor named as a joint in

In [24]:

# ---------------------------------------------------------
# Step 2: Initialize the web-search agent
# ---------------------------------------------------------
# Goal:
#   Connect your `web_search` tool to a language model
#   so the agent can search and reason over real data.
#
# Steps:
#   1. Import `initialize_agent` and `AgentType`.
#   2. Create an LLM (e.g., ChatOllama).
#   3. Add your `web_search` tool to the tools list.
#   4. Initialize the agent using: initialize_agent
#   5. Keep `verbose=True` to observe reasoning steps.
#
# Expected:
#   The agent should be ready to accept user queries
#   and use your web search tool when needed.
# ---------------------------------------------------------
from langchain_community.chat_models import ChatOllama
from langchain.agents import initialize_agent, AgentType

# Create an LLM instance with the correct base URL for dev container
llm = ChatOllama(
    model="gemma3:4b",
    temperature=0,
    base_url="http://host.docker.internal:11434"
)

# Add web_search tool to the tools list
tools = [web_search]

# Initialize the agent with ReAct reasoning
web_agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True
)

print("Web-search agent initialized successfully!")
print(f"Available tools: {[tool.name for tool in tools]}")

Web-search agent initialized successfully!
Available tools: ['web_search']



Let’s see the agent's output in action with a real example.


In [25]:

# ---------------------------------------------------------
# Step 3: Test your Ask-the-Web agent
# ---------------------------------------------------------
# Goal:
#   Verify that the agent can search the web and return
#   a summarized answer based on real results.
#
# Steps:
#   1. Ask a natural question that requires live information,
#      for example: "What are the current events in San Francisco this week?"
#   2. Call agent.
#
# Expected:
#   The agent should call `web_search`, retrieve results,
#   and generate a short summary response.
# ---------------------------------------------------------

# Test with a question requiring live information
test_question = "What are the latest developments in artificial intelligence in 2025?"

print("=" * 60)
print(f"USER QUESTION: {test_question}")
print("=" * 60)

# Invoke the web agent
response = web_agent.invoke({"input": test_question})

# Display the final answer
print("\n" + "=" * 60)
print("FINAL ANSWER:")
print("=" * 60)
print(response["output"])

USER QUESTION: What are the latest developments in artificial intelligence in 2025?


> Entering new AgentExecutor chain...
Action: ```{
  "action": "web_search",
  "action_input": "AI developments 2025"
}```
Observation: 1. Key AI Developments in 2025 : Shaping Our Future
   2025 marks a pivotal moment: AI is no longer futuristic—it’s practical, powerful, and omnipresent. Yet, its potential must be balanced with ethical responsibility and human oversight.
   URL: https://www.linkedin.com/pulse/key-ai-developments-2025-shaping-our-future-rama-prabha-d-hnxfc

2. The 2025 AI Gold Rush: 9 startups poised to redefine the... | Medium
   Healthcare is a goldmine for AI — but also a regulatory minefield. In 2025 , healthcare startups will focus less on diagnosis and more on automation of care workflows.
   URL: https://medium.com/artificial-intelligence-ai-uncovered/the-2025-ai-gold-rush-9-startups-poised-to-redefine-the-future-783e688d2a61

3. Human Development Report 2025 | Human Developmen


## 6- A minimal UI
This project includes a simple **React** front end that sends the user’s question to a FastAPI back end and streams the agent’s response in real time. To run the UI:

1- Open a terminal and start the Ollama server: `ollama serve`.

2- In a second terminal, navigate to the frontend folder and install dependencies:`npm install`.

3- In the same terminal, start the FastAPI back‑end: `uvicorn app:app --reload --port 8000`

4- Open a third terminal, stay in the frontend folder, and start the React dev server: `npm run dev`

5- Visit `http://localhost:5173/` in your browser.



## 🎉 Congratulations!

* You have built a **web‑enabled agent**: tool calling → JSON schema → LangChain ReAct → web search → simple UI.
* Try adding more tools, such as news or finance APIs.
* Experiment with multiple tools, different models, and measure accuracy vs. hallucination.


👏 **Great job!** Take a moment to celebrate. The techniques you implemented here power many production agents and chatbots.